In [13]:
# imports packages
from __future__ import print_function 
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import pandas as pd
#import csv
import random
import scipy.integrate as integrate
from numpy import genfromtxt
from random import randrange
from numpy.polynomial import Polynomial as P
from IPython.display import Latex


# Convolution

In [14]:
from scipy.ndimage.filters import convolve

In [2]:
# Create halo array from halo table for convolution

def create_halo_array_for_convolution(pdHalos,M_min,M_max,logchunks):
    halos = haloArray_minmax(pdHalos,M_min,M_max)
    df = halos.sort_values(by='Mvir',ascending=True)
    sorted_haloMasses = df['Mvir'].values
    histH, binsH = np.histogram(sorted_haloMasses,bins=np.logspace(np.log10(M_min),np.log10(M_max), logchunks))
    binz=np.append(0,histH.cumsum())
    
    return df,binz,binsH

In [3]:
# Convert custom 3D function to 2D by integrating over z
def func3Dto2D(f,x,y,Rvir):
    return integrate.quad(f,-(Rvir**2-(x**2+y**2))**.5,(Rvir**2-(x**2+y**2))**.5,args=(x,y))[0]


In [11]:
# Project NFW profile from 3D to 2D
def NFW2D(x,y,rho_nought,R_s,Rvir):
    offset=float(.1)
    return integrate.quad(lambda x, y, z: rho_nought/(((offset+(x**2+y**2+ z**2)**.5)/R_s)*(1+((x**2+y**2+ z**2)**.5)/R_s)**2),-1*np.real((Rvir**2-(x**2+y**2))**.5),np.real((Rvir**2-(x**2+y**2))**.5),args=(x,y))[0]



In [9]:
# Function creates a smaller grid from a larger grid by smoothing

def smoothfield(big, nbig,nsmall):
    
    roll = int((nbig/nsmall)/2)
    y = np.roll(big, roll, axis=0)  #so that y grid takes 2 cells behind and 2 in front
    y = np.roll(y, roll, axis=1)

    small = y.reshape([nsmall, nbig//nsmall, nsmall, nbig//nsmall]).mean(3).mean(1)
    return small

In [18]:
# Function subtracts halo field from density field

def removeConvolvedHalos(densityField,haloField):
    
    return densityField-haloField


# Gaussian and sinc smoothing of the halo field

In [12]:
# Description:


def gauss_sinc_smoothing(smoothing_array,sigma_gauss,width_sinc,resolution):
    
    func = smoothing_array
    fft_func = np.fft.fft2(func)

    # sigma is the variance of the gaussian. sigma**2 = 16 or 3*sigma**2 = 16
    sigma = sigma_gauss

    # nsinc is the full width of the box function. So 4 means two boxes on either side of the point.
    nsinc = width_sinc

    # wavenumbers for the transforms
    kx = (np.fft.fftfreq(1024*resolution))
    ky = kx[:,np.newaxis]

    #     ky = (np.fft.fftfreq(1024*resolution))

    prod = np.zeros([1024*resolution,1024*resolution],dtype=complex)

#     for i in range(1024*resolution):
#         for j in range(1024*resolution):

#             # correct functions
#             prod[i,j] = fft_func[i,j]*np.exp(-(sigma**2) *2*np.pi**2* (kx[i]**2 + ky[j]**2 ))
            
#             prod[i,j] =  prod[i,j]*(np.sinc(kx[i]*nsinc*resolution)*np.sinc(ky[j]*nsinc*resolution))**2


    prod = fft_func*np.exp(-(sigma**2) *2*np.pi**2* (kx**2 + ky**2 )) #* (np.sinc(kx*nsinc*resolution)*np.sinc(ky*nsinc*resolution))**2
    result = np.fft.ifft2(prod)
    
    return result.real



# Halo subtraction

This function subtracts the halos from the density field

In [3]:
# The function convolves halos

# arguments: 
# haloArray: array of halos
# resolution: 1024 or 2048
# chunks: no of chunks the halo array has been divided into -1
# bins: array cumulative sum of halos in different mass bins
# profile: tophat, NFW etc
# scaling_radius: scale radius for tophat halos

def subtract_halos(haloArray,mass_binz,resolution,chunks,bins,profile,scaling_radius,redshift):
    
    
    df = haloArray
    no_cells = 1024
    cellsize = L/(1024) 
    
    
    # array of halo masses and radii
    Mvir_avg = np.zeros(chunks)
    conv_rad = np.zeros(chunks)
    Rvir_avg = np.zeros(chunks)

    # convolution mask array
    convolution =np.zeros([chunks,no_cells,no_cells])
    
    # creating a coarse map out of a fine mask
    
    # fine mask
    fine_mask_len = 10
    fine_lower= -1*fine_mask_len
    fine_upper= fine_mask_len
    y,x = np.ogrid[fine_lower: fine_upper, fine_lower:fine_upper]
    
    # coarse mask
    scale_down = 2  # making the grid coarser
    smooth_r=2
    
    
    coarse_mask_len = int(fine_mask_len/scale_down)
    fine_mask= np.zeros([2*coarse_mask_len,2*coarse_mask_len])
    
    # loops through the list of dataframes each ordered by ascending mass
    
    for j in range(0,chunks):
        Mvir_avg[j] = np.mean((df['Mvir'][bins[j]:bins[j+1]]))/h
        conv_rad[j] = (1+redshift)*((Mvir_avg[j])**(1/3) / Rvir_den(redshift)) # comoving radius
        
        
        # NFW 
        # add redshift to the function above
        R_s=0
        rho_nought=0
        
        R_s = conv_rad[j]/(halo_conc(redshift,Mvir_avg[j])*cellsize)  
        rho_nought = rho_0(redshift,Mvir_avg[j],R_s)
        
        
        
        # Why are we multiplying by scale_down
        if profile == 'tophat':
            r = (x**2+y**2)**.5
            fine_mask = r <= (scaling_radius*scale_down*conv_rad[j]/cellsize) 
            fine_mask=fine_mask.astype(float)

            
        # spherical tophat
        elif profile == 'tophat_spherical':
            r = (x**2+y**2)**.5
            fine_mask = r <= (scaling_radius*scale_down*conv_rad[j]/cellsize)
            
            fine_mask=fine_mask.astype(float)
#             mask5 = mask5* (2*(((scaling_radius*scale_down*conv_rad[j]/cellsize)**2-(r**2))**2)**.25)
            
            Rv= (scaling_radius*scale_down*conv_rad[j]/cellsize)
            fine_mask = fine_mask* ((((1)**2-((r/Rv)**2))**2)**.25)
            
        
        
        elif profile == 'NFW':
            vec_integral = np.vectorize(NFW2D)
            fine_mask =vec_integral(x,y,rho_nought,R_s,conv_rad[j])

            r=(x**2+y**2)**.5 # * scale_down
            
            fine_mask[r> scale_down*conv_rad[j]/cellsize] =0 
            
            fine_mask=fine_mask.astype(float)
        
        
        elif profile == 'custom':
            
            # Functions for profile
            # Currently hardcoded but should allow the user to provide the function as inputs
            
            f1_1 = lambda x,y,z: 1/(x**2+y**2+z**2+.5)**.5
            f1_2 = lambda x,y,z: 1/(x**2+y**2+z**2+.5)
            
            # Radius of first profile:
            R= (scaling_radius*scale_down*conv_rad[j]/cellsize)/2 
            
            
            
            
            
            vec_integral = np.vectorize(func3Dto2D)
            
            mask1 = x**2+y**2 < (R/2)**2
            mask1 = mask1.astype(float)*vec_integral(f1_1,x,y,R/2)

            mask2_1 = x**2+y**2 >= (R/2)**2
            mask2_1= mask2_1.astype(float)
            mask2_2 = x**2+y**2 <= (R)**2
            mask2_2= mask2_2.astype(float)
            mask2=mask2_1*mask2_2
            mask2 = mask2*vec_integral(f1_2,x,y,R)


            fine_mask=mask1+mask2
            
        
        elif profile == 'custom_tophat':
            
            # Functions for profile
            # Currently hardcoded but should allow the user to provide the function as inputs
            
            # Radius of first profile:
            R= (scaling_radius*scale_down*conv_rad[j]/cellsize)
            
            
            f1_1 = lambda x,y,z: np.exp(-((x**2+y**2+z**2)/R**2)**30)
            
            vec_integral = np.vectorize(func3Dto2D)
            
            mask1 = vec_integral(f1_1,x,y,R)

            fine_mask=mask1
        
        
        
        
        # Smoothing method: reshaping
        # Generating coarse grid from fine grid: reshape method
        nbig = fine_mask_len*2
        nsmall = int(nbig/scale_down)
        coarse_mask = fine_mask.reshape([nsmall, nbig//nsmall, nsmall, nbig//nsmall]).mean(3).mean(1)
        
        

        
        # Area of cells needed for normalization
        totalcellArea4=0
        totalcellArea4 = sum(sum(coarse_mask))* ((cellsize)**2)
        


        # populate array with halos
        halo_cell_pos = np.zeros([no_cells,no_cells])    
        
        # The coordinates are being multiplied by 4 to yield the halo coordinates on the 1024 grid
        ix = ((((np.around(4*((df[bins[j]:bins[j+1]]['x'].values)/(250/256))))))%(1024)).astype(int)
        iy = ((((np.around(4*((df[bins[j]:bins[j+1]]['y'].values)/(250/256))))))%(1024)).astype(int)
        
        
        xy=(ix,iy)

        # issue: the method does not add repeated coordinates
        halo_cell_pos[xy] += 1
        
        
        
        # convolve the mask and the halo positions
        
        convolution[j,:,:] = (Mvir_avg[j]/(totalcellArea4))*convolve(halo_cell_pos,coarse_mask)
        
        
    
    return (convolution.sum(0))*(Mpc**-3 *10**6)*nPS*(OmegaB/OmegaM), conv_rad, Rvir_avg, fine_mask, coarse_mask,halo_cell_pos

    


# Halo addition

In [1]:
# Status: This is the latest convolution function

# The function convolves halos

# arguments: 
# haloArray: array of halos
# resolution: 1024 or 2048
# chunks: no of chunks the halo array has been divided into -1
# bins: array cumulative sum of halos in different mass bins
# profile: tophat, NFW etc
# scaling_radius: scale radius for tophat halos

def add_halos(haloArray,mass_binz,resolution,chunks,bins,profile,scaling_radius,redshift):
    
    
    df = haloArray
    no_cells = 1024* resolution
    cellsize = L/(1024*resolution) 
    
    
    # array of halo masses and radii
    Mvir_avg = np.zeros(chunks)
    conv_rad = np.zeros(chunks)
    Rvir_avg = np.zeros(chunks)

    # convolution mask array
    convolution =np.zeros([chunks,no_cells,no_cells])
    
    
    # creating a coarse map out of a fine mask
    
    # fine mask
    # fine mask size has to correspond to the size of the mask that I eventually trim
    fine_mask_len = 20*resolution  
    fine_lower= -1*fine_mask_len
    fine_upper= fine_mask_len
    y,x = np.ogrid[fine_lower: fine_upper, fine_lower:fine_upper]
    
    
    # coarse mask
    scale_down = 2  # making the grid coarser
    smooth_r=2
    
    
#     coarse_mask_len = int(fine_mask_len/scale_down)
    fine_mask= np.zeros([fine_mask_len,fine_mask_len])
# #     fine_mask= np.zeros([2*coarse_mask_len,2*coarse_mask_len])
    
    
    
    # store all profile masks
    nbig = fine_mask_len*2
    nsmall = int(nbig/scale_down)
    addition_masks =np.zeros([chunks,nsmall,nsmall])
    
    # loops through the list of dataframes each ordered by ascending mass
    
    for j in range(0,chunks):
        Mvir_avg[j] = np.mean((df['Mvir'][bins[j]:bins[j+1]]))/h
        conv_rad[j] = (1+redshift)*((Mvir_avg[j])**(1/3) / Rvir_den(redshift)) # comoving radius
      
            
        
        # NFW 
        # add redshift to the function above
        R_s=0
        rho_nought=0
        
        R_s= conv_rad[j]/(halo_conc(redshift,Mvir_avg[j])*cellsize)  
        rho_nought = rho_0(redshift,Mvir_avg[j],R_s)
        
        
        
        # Why are we multiplying by scale_down
        if profile == 'tophat':
            r = (x**2+y**2)**.5
            fine_mask = r <= (scaling_radius*scale_down*conv_rad[j]/cellsize) 
            fine_mask=fine_mask.astype(float)

            
        # spherical tophat
        elif profile == 'tophat_spherical':
            r = (x**2+y**2)**.5
            fine_mask = r <= (scaling_radius*scale_down*conv_rad[j]/cellsize)
            
            fine_mask=fine_mask.astype(float)
#             mask5 = mask5* (2*(((scaling_radius*scale_down*conv_rad[j]/cellsize)**2-(r**2))**2)**.25)
            
            Rv= (scaling_radius*scale_down*conv_rad[j]/cellsize)
            fine_mask = fine_mask* ((((1)**2-((r/Rv)**2))**2)**.25)
            
        
        
        elif profile == 'NFW':
            
            vec_integral = np.vectorize(NFW2D)
            fine_mask =vec_integral(x,y,rho_nought,R_s,conv_rad[j]/cellsize)
            
            r=(x**2+y**2)**.5 # * scale_down
            
            fine_mask=fine_mask.astype(float)
            fine_mask[r> scale_down*conv_rad[j]/cellsize] =0 
            
            
        
        
        elif profile == 'custom':
            
            # Functions for profile
            # Currently hardcoded but should allow the user to provide the function as inputs
            
            f1_1 = lambda x,y,z: 1/(x**2+y**2+z**2+.5)**.5
            f1_2 = lambda x,y,z: 1/(x**2+y**2+z**2+.5)
            
            # Radius of first profile:
            R= (scaling_radius*scale_down*conv_rad[j]/cellsize)/2 
            
            
            
            
            
            vec_integral = np.vectorize(func3Dto2D)
            
            mask1 = x**2+y**2 < (R/2)**2
            mask1 = mask1.astype(float)*vec_integral(f1_1,x,y,R/2)

            mask2_1 = x**2+y**2 >= (R/2)**2
            mask2_1= mask2_1.astype(float)
            mask2_2 = x**2+y**2 <= (R)**2
            mask2_2= mask2_2.astype(float)
            mask2=mask2_1*mask2_2
            mask2 = mask2*vec_integral(f1_2,x,y,R)


            fine_mask=mask1+mask2
            
        
        elif profile == 'custom_tophat':
            
            # Functions for profile
            # Currently hardcoded but should allow the user to provide the function as inputs
            
            # Radius of first profile:
            R= (scaling_radius*scale_down*conv_rad[j]/cellsize)
            
            
            f1_1 = lambda x,y,z: np.exp(-((x**2+y**2+z**2)/R**2)**30)
            
            vec_integral = np.vectorize(func3Dto2D)
            
            mask1 = vec_integral(f1_1,x,y,R)

            fine_mask=mask1
        
        
        ## testing code to add Fire simulation halos
        elif profile == "fire":
        
            %run gasProfile_Fire.ipynb
            fine_mask = rhogasFire(Mvir_avg[j],conv_rad[j], redshift, adjustmentfactor,resolution)[2]
        
        
        
        # Precipitation model
        elif profile == "precipitation":
        
            %run gasProfile_precipitation.ipynb
            
#             fine_mask = rhogasFire(Mvir_avg[j],conv_rad[j], redshift, adjustmentfactor,resolution)[2]
            fine_mask = nePercipitation(np.log10(Mvir_avg[j]),1000*conv_rad[j],resolution,redshift)[1]
        
        
        elif profile == "2RVSTH_and_NFW_13.5":
            if Mvir_avg[j] <= 10**13.5:
                
                r = (x**2+y**2)**.5
                
                # Don't hardcode scaling radius, change later
                fine_mask = r <= ((2*scaling_radius)*scale_down*conv_rad[j]/cellsize)
            
                fine_mask=fine_mask.astype(float)
#             mask5 = mask5* (2*(((scaling_radius*scale_down*conv_rad[j]/cellsize)**2-(r**2))**2)**.25)
            
                Rv= (scaling_radius*scale_down*conv_rad[j]/cellsize)
                fine_mask = fine_mask* ((((1)**2-((r/Rv)**2))**2)**.25)
            
            elif Mvir_avg[j] > 10**13.5:
                vec_integral = np.vectorize(NFW2D)
                fine_mask =vec_integral(x,y,rho_nought,R_s,conv_rad[j]/cellsize)

                r=(x**2+y**2)**.5 # * scale_down

                fine_mask=fine_mask.astype(float)
                fine_mask[r> scale_down*conv_rad[j]/cellsize] =0 
        
        
        elif profile == "2RVSTH_and_NFW_13":
            if Mvir_avg[j] <= 10**13:
                
                r = (x**2+y**2)**.5
                
                # Don't hardcode scaling radius, change later
                fine_mask = r <= ((2*scaling_radius)*scale_down*conv_rad[j]/cellsize)
            
                fine_mask=fine_mask.astype(float)
#             mask5 = mask5* (2*(((scaling_radius*scale_down*conv_rad[j]/cellsize)**2-(r**2))**2)**.25)
            
                Rv= (scaling_radius*scale_down*conv_rad[j]/cellsize)
                fine_mask = fine_mask* ((((1)**2-((r/Rv)**2))**2)**.25)
            
            elif Mvir_avg[j] > 10**13:
                vec_integral = np.vectorize(NFW2D)
                fine_mask =vec_integral(x,y,rho_nought,R_s,conv_rad[j]/cellsize)

                r=(x**2+y**2)**.5 # * scale_down

                fine_mask=fine_mask.astype(float)
                fine_mask[r> scale_down*conv_rad[j]/cellsize] =0 
        
        
        
        elif profile == "2RVSTH_and_NFW_12.5":
            if Mvir_avg[j] <= 10**12.5:
                
                r = (x**2+y**2)**.5
                
                # Don't hardcode scaling radius, change later
                fine_mask = r <= ((2*scaling_radius)*scale_down*conv_rad[j]/cellsize)
            
                fine_mask=fine_mask.astype(float)
#             mask5 = mask5* (2*(((scaling_radius*scale_down*conv_rad[j]/cellsize)**2-(r**2))**2)**.25)
            
                Rv= (scaling_radius*scale_down*conv_rad[j]/cellsize)
                fine_mask = fine_mask* ((((1)**2-((r/Rv)**2))**2)**.25)
            
            elif Mvir_avg[j] > 10**12.5:
                vec_integral = np.vectorize(NFW2D)
                fine_mask =vec_integral(x,y,rho_nought,R_s,conv_rad[j]/cellsize)

                r=(x**2+y**2)**.5 # * scale_down

                fine_mask=fine_mask.astype(float)
                fine_mask[r> scale_down*conv_rad[j]/cellsize] =0 
        
        
        
        elif profile == "2RVSTH_and_NFW_12":
            if Mvir_avg[j] <= 10**12:
                
                r = (x**2+y**2)**.5
                
                # Don't hardcode scaling radius, change later
                fine_mask = r <= ((2*scaling_radius)*scale_down*conv_rad[j]/cellsize)
            
                fine_mask=fine_mask.astype(float)
#             mask5 = mask5* (2*(((scaling_radius*scale_down*conv_rad[j]/cellsize)**2-(r**2))**2)**.25)
            
                Rv= (scaling_radius*scale_down*conv_rad[j]/cellsize)
                fine_mask = fine_mask* ((((1)**2-((r/Rv)**2))**2)**.25)
            
            elif Mvir_avg[j] > 10**12:
                vec_integral = np.vectorize(NFW2D)
                fine_mask =vec_integral(x,y,rho_nought,R_s,conv_rad[j]/cellsize)

                r=(x**2+y**2)**.5 # * scale_down

                fine_mask=fine_mask.astype(float)
                fine_mask[r> scale_down*conv_rad[j]/cellsize] =0 
        
        
        
        elif profile == "2RVSTH_and_NFW_11.5":
            if Mvir_avg[j] <= 10**11.5:
                
                r = (x**2+y**2)**.5
                
                # Don't hardcode scaling radius, change later
                fine_mask = r <= ((2*scaling_radius)*scale_down*conv_rad[j]/cellsize)
            
                fine_mask=fine_mask.astype(float)
#             mask5 = mask5* (2*(((scaling_radius*scale_down*conv_rad[j]/cellsize)**2-(r**2))**2)**.25)
            
                Rv= (scaling_radius*scale_down*conv_rad[j]/cellsize)
                fine_mask = fine_mask* ((((1)**2-((r/Rv)**2))**2)**.25)
            
            elif Mvir_avg[j] > 10**11.5:
                vec_integral = np.vectorize(NFW2D)
                fine_mask =vec_integral(x,y,rho_nought,R_s,conv_rad[j]/cellsize)

                r=(x**2+y**2)**.5 # * scale_down

                fine_mask=fine_mask.astype(float)
                fine_mask[r> scale_down*conv_rad[j]/cellsize] =0 
                
                
                
        
        elif profile == "2RVSTH_and_NFW_11":
            if Mvir_avg[j] <= 10**11:
                
                r = (x**2+y**2)**.5
                
                # Don't hardcode scaling radius, change later
                fine_mask = r <= ((2*scaling_radius)*scale_down*conv_rad[j]/cellsize)
            
                fine_mask=fine_mask.astype(float)
#             mask5 = mask5* (2*(((scaling_radius*scale_down*conv_rad[j]/cellsize)**2-(r**2))**2)**.25)
            
                Rv= (scaling_radius*scale_down*conv_rad[j]/cellsize)
                fine_mask = fine_mask* ((((1)**2-((r/Rv)**2))**2)**.25)
            
            elif Mvir_avg[j] > 10**11:
                vec_integral = np.vectorize(NFW2D)
                fine_mask =vec_integral(x,y,rho_nought,R_s,conv_rad[j]/cellsize)

                r=(x**2+y**2)**.5 # * scale_down

                fine_mask=fine_mask.astype(float)
                fine_mask[r> scale_down*conv_rad[j]/cellsize] =0        
                
                
                
        
        
        # Smoothing method: reshaping
        # Generating coarse grid from fine grid: reshape method
        
        
        nsmall = int(nbig/scale_down)
        
        coarse_mask = fine_mask.reshape([nsmall, nbig//nsmall, nsmall, nbig//nsmall]).mean(3).mean(1)
        
        
        
        
        # Area of cells needed for normalization
        totalcellArea4=0
        totalcellArea4 = sum(sum(coarse_mask))* ((cellsize)**2)
        


        # populate array with halos
        halo_cell_pos = np.zeros([no_cells,no_cells])    
        
        # The coordinates are being multiplied by 4 to yield the halo coordinates on the 1024 grid
        ix = ((((np.around(4*resolution*((df[bins[j]:bins[j+1]]['x'].values)/(250/256))))))%(resolution*1024)).astype(int)
        iy = ((((np.around(4*resolution*((df[bins[j]:bins[j+1]]['y'].values)/(250/256))))))%(resolution*1024)).astype(int)
        
        
        xy=(ix,iy)

        # issue: the method does not add repeated coordinates
        halo_cell_pos[xy] += 1
        
        
        
        # convolve the mask and the halo positions
        
        convolution[j,:,:] = (Mvir_avg[j]/(totalcellArea4))*convolve(halo_cell_pos,coarse_mask)
        
        
        
        # store addition masks
        addition_masks[j,:,:]= (Mvir_avg[j]/(totalcellArea4))*(Mpc**-3 *10**6)*nPS*(OmegaB/OmegaM)*coarse_mask
        

    
    return (convolution.sum(0))*(Mpc**-3 *10**6)*nPS*(OmegaB/OmegaM), conv_rad, Rvir_avg, fine_mask, coarse_mask,halo_cell_pos,addition_masks,Mvir_avg

    


# Halos removed field

This function combines many steps of subtraction and smoothing to yield a density field from which halos have been removed


In [4]:


def halos_removed_field(current_halo_file,min_mass,max_mass,density_field,den_grid_size,redshift,log_bins,subtraction_halo_profile,scaling_radius,resolution,sigma_gauss,width_sinc):
    
    
    halo_array_for_convolution = create_halo_array_for_convolution(current_halo_file,min_mass,max_mass,log_bins)
    df= halo_array_for_convolution[0]
    binz= halo_array_for_convolution[1]
    mass_binz = halo_array_for_convolution[2]
    
   
    
    
    # convolve halos
    subtraction_profile = subtract_halos(df,mass_binz,resolution,len(binz)-1,binz,subtraction_halo_profile,scaling_radius,redshift)[0]
    
   
    
    subtraction_profile_smooth = gauss_sinc_smoothing(subtraction_profile,sigma_gauss,width_sinc,1)
    
   
    
    # create coarse grid
    subtracted_coarse= smoothfield(subtraction_profile_smooth,1024,den_grid_size)
    
   
    # remove halos from the density field
    halos_removed_coarse=removeConvolvedHalos(density_field,subtracted_coarse)
    
    return halos_removed_coarse,subtracted_coarse

# Function subtracts and adds halos



In [5]:
def convolution_all_steps_final(current_halo_file,min_mass,max_mass,density_field,den_grid_size,redshift,log_bins,halos_removed_coarse,
                       addition_halo_profile,scaling_radius,resolution,sigma_gauss,width_sinc):
    t1 = time.time()
    
    # setup inputs for convolution
    
    halo_array_for_convolution = create_halo_array_for_convolution(current_halo_file,min_mass,max_mass,log_bins)
    df= halo_array_for_convolution[0]
    binz= halo_array_for_convolution[1]
    mass_binz = halo_array_for_convolution[2]
    
    t2 = time.time()
    
    
    # convolve halos for adding back
    addition_profile_initial=add_halos(df,mass_binz,resolution,len(binz)-1,binz,addition_halo_profile,scaling_radius,redshift)
    addition_profile = addition_profile_initial[0]
    addition_profile_masks=addition_profile_initial[6]
    
    
    t7 = time.time()
    
    # add halos to the subtracted field
    halosremoved_fine = (np.repeat((np.repeat(halos_removed_coarse,(1024/den_grid_size)*resolution,axis=0)),(1024/den_grid_size)*resolution,axis=1))
    roll_by = int((addition_profile.shape[0]/den_grid_size)/2)
    halosremoved_fine = np.roll(halosremoved_fine, -1*roll_by, axis=0)
    halosremoved_fine = np.roll(halosremoved_fine, -1*roll_by, axis=1)
    halos_added = addition_profile +  halosremoved_fine
    
    t8 = time.time()
    
    virial_rad = addition_profile_initial[1]
    halo_masses = addition_profile_initial[7]
    
    return halos_added, halosremoved_fine,addition_profile,addition_profile_masks,halos_removed_coarse,virial_rad,halo_masses
    

# Multiple redshifts convolution

In [77]:

def halo_subtraction_addition(all_den_fields,den_grid_size,RS_array,min_mass,max_mass,log_bins,subtraction_halo_profile,
                             addition_halo_profile,scaling_radius,resolution):

    # Details of halo profiles

    # sigma is the variance of the gaussian. sigma**2 = 16 or 3*sigma**2 = 16
    sigma_gauss = 4*resolution/np.sqrt(3)

    # nsinc is the full width of the box function. So 4 means two boxes on either side of the point.
    width_sinc = 4

    

    halos_reAdded = np.zeros([len(RS_array),1024*resolution,1024*resolution])
    
    
    halos_subtraction_coarse = np.zeros([len(RS_array),den_grid_size,den_grid_size])
    halo_field = np.zeros([len(RS_array),1024*resolution,1024*resolution])
    halo_masks = np.zeros([len(RS_array),int(log_bins-1),20*resolution,20*resolution]) # This should be the same as fine_mask_len in add_halos function
    halos_removed_fields = np.zeros([len(RS_array),den_grid_size,den_grid_size]) # This should be the same as fine_mask_len in add_halos function
    
    
    
    for i in range(0, len(RS_array)):
#         current_halo_file = extract_halos(RS_array[i])
        current_halo_file = extract_halos(i)
        halos_removed = halos_removed_field(current_halo_file,min_mass,max_mass,all_den_fields[i,:,:],den_grid_size,RS_array[i],log_bins,subtraction_halo_profile,scaling_radius,resolution,sigma_gauss,width_sinc)
        
                
        conv_all_steps = convolution_all_steps_final(current_halo_file,min_mass,max_mass,all_den_fields[i,:,:],den_grid_size,RS_array[i],log_bins,halos_removed[0],
                           addition_halo_profile,scaling_radius,resolution,sigma_gauss,width_sinc)
        
        
        halos_reAdded[i,:,:] = conv_all_steps[0]
        
        
        halos_subtraction_coarse[i,:,:] = halos_removed[1]
        halo_field[i,:,:] = conv_all_steps[2]
        halo_masks[i,:,:,:]= conv_all_steps[3]
        halos_removed_fields[i,:,:] =  halos_removed[0]
        
        
        
        
    # returns halo array and masks used to add halos back
    return halos_reAdded,halo_masks,halos_subtraction_coarse,halo_field,halos_removed_fields, conv_all_steps[5],conv_all_steps[6]
    

# Master function 


In [ ]:
# This function runs all the functions above functions at once

# Outputs:histograms,halos-readded field, halo addition masks, halos subtraction coarse, halo addition field, 
         #halos removed field, stacked halo field

def hist_profile(all_den_fields,den_grid_size,RS_array,min_mass,max_mass,
                                       log_bins,subtraction_halo_profile,addition_halo_profile,scaling_radius,resolution):
    
    
    # halo array
    t = halo_subtraction_addition(all_den_fields,den_grid_size,RS_array,min_mass,max_mass,
                                       log_bins,subtraction_halo_profile,addition_halo_profile,scaling_radius,resolution)
    # Halos-readded field
    t1=t[0]
    
    # Halo addition masks
    t2=t[1]
    
    # Halos subtraction coarse
    t3=t[2]
    
    # Halo addition field
    t4=t[3]
    
    # Halos removed field
    t5 = t[4]
    
    if len(RS_array)==1:
        t6=t1
    
    else:
        t6= stack_all_arrays(t1,RS_array)
        
    t7= 1#create_histograms(t6,resolution)
    
    t8 = t[5]
    t9 = t[6]
    
    # Outputs: 
    # 1s,halos-readded field, halo addition masks, halos subtraction coarse, halo addition field, halos removed field, stacked halo field
    return t7,t1,t2,t3,t4,t5,t6,t8,t9